# Calibration

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import torch
import wandb

from calibration import data
from calibration import dist
from calibration import method
from calibration import pit
from calibration import plot

In [ ]:
def param2pdf(weight, mean, variance):
    return lambda x: dist.pdf_gaussian_mixture(x, weight, mean, variance)

def y2pdf(w, s, v1, v2):
    weight = torch.tensor([w, 1 - w])
    mean = torch.tensor([-s / 2, s / 2])
    variance = torch.tensor([v1, v2])
    return param2pdf(weight, mean, variance)

## Interpreter

In [ ]:
checkpoint = torch.load("models/skilled-monkey-956.pt")
hyperparams = checkpoint["hyperparams"]
interpreter = method.MDN(inputs=hyperparams["bins"],
                neurons=hyperparams["neurons"],
                components=hyperparams["m"])
interpreter.load_state_dict(checkpoint["model_state_dict"])
interpreter

## Synthetic data set

In [ ]:
TESTS = 1000
torch.manual_seed(78)
testset = pit.PITDataset(TESTS)

In [ ]:
refset = pit.PITReference()
len(refset)

In [ ]:
pred_mdn = method.predict(interpreter, testset.X)

In [ ]:
distances = euclidean_distances(testset.X, refset.X)
js = distances.argmin(axis=1)
for y, weight, mean, variance, neighbour in zip(testset.annotation,
                                          *pred_mdn,
                                          refset.annotation[js]):
    _, ax = plt.subplots()
    plot.density(ax, y2pdf(*y), label="data generating")
    plot.density(ax, param2pdf(weight, mean, variance), label="MDN")
    plot.density(ax, y2pdf(*neighbour), label="nearest neighbor")
    ax.legend()
    plt.show()
    plt.close()

In [ ]:
nll_mdn = dist.nll_gaussian_mixture(testset.y, *pred_mdn).mean()
nll_mdn.item()

In [ ]:
steps = range(5, 18)
nll_neighbour = []
for s in steps:
    refset = pit.PITReference(steps=s)
    distances = euclidean_distances(testset.X, refset.X)
    js = distances.argmin(axis=1)
    neighbour = refset.annotation[js]
    weight = torch.stack([neighbour[:, 0], 1 - neighbour[:, 0]], dim=1)
    mean = torch.stack((-neighbour[:, 1] / 2, neighbour[:, 1] / 2), dim=1)
    variance = neighbour[:, 2:]
    nll = dist.nll_gaussian_mixture(testset.y, weight, mean, variance)
    nll_neighbour.append(nll.mean().item())
    print(f"{s:2d} {nll_neighbour[-1]:f} {s ** 4}")

_, ax = plt.subplots()
size = [s ** 4 for s in steps]
ax.scatter(size, nll_neighbour, label="$k$-NN")
ax.axhline(nll_mdn, label="our MDN")
ax.set_xlabel("size of reference set")
ax.set_ylabel("NLL")
ax.legend()
nll_neighbour

In [ ]:
df = pd.DataFrame.from_dict({"steps": steps,
                             "size": size,
                             "nll": nll_neighbour})
df = df.set_index("steps")
df.to_csv("data/neighbour.csv")
df

## Uniform PIT histogram

In [ ]:
pit_hist_uniform = torch.full((pit.BINS, ), 1 / pit.BINS)
pred_uniform = method.predict(interpreter, pit_hist_uniform)
_, ax = plt.subplots()
plot.density(ax, dist.pdf_gaussian, label="data-generating")
plot.density(ax, param2pdf(*pred_uniform), label="predicted data-generating")
ax.legend()

## UCI ML repository data sets

TODO list:
1. more data sets: power, protein, year
1. what hyperparameters to use?
1. confidence intervals or REM
1. other methods: MC dropout?, concrete dropout?, Gaussian processes?

In [ ]:
def plot_interpretation(pit_hist, pred):
    _, (ax1, ax2) = plt.subplots(nrows=2)
    plot.pit_hist(ax1, pit_hist, label="true")
    plot.pit_hist(ax1,
                  pit.pit_hist(pit.pit_gaussian(dist.sample_gaussian_mixture(*pred))),
                  label="predicted")
    ax1.legend()
    plot.density(ax2, dist.pdf_gaussian, label="predictive")
    plot.density(ax2, param2pdf(*pred), label="predicted data-generating")
    ax2.legend()

def load_dn(modelfile):
    checkpoint = torch.load(modelfile)
    hyperparams = checkpoint["hyperparams"]
    dn = method.MDN(inputs=hyperparams["inputs"],
                    neurons=hyperparams["neurons"],
                    components=hyperparams["m"])
    dn.load_state_dict(checkpoint["model_state_dict"])
    return dn

### Metrics

In [ ]:
api = wandb.Api()
runs = api.runs("podondra/calibration")

keys = ["dataname", "seed", "method"]
data, names = [], []
for run in runs:
    names.append(run.name)
    dictionary = {k: run.config[k] for k in keys}
    dictionary["nll"] = run.summary["test.nll"]
    dictionary["crps"] = run.summary["test.crps"]
    data.append(dictionary)

df = pd.DataFrame(data=data, index=names)
df

In [ ]:
gdf = df.groupby(["dataname", "method"])

In [ ]:
gdf["nll"].agg(["mean", "sem"])

In [ ]:
gdf["crps"].agg(["mean", "sem"])

### Protein

In [ ]:
X, y = data.protein()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=33)
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.1, random_state=79)

#### Density network

In [ ]:
dn = load_dn("models/driven-serenity-33.pt")

y_dn = inverse_transform(method.predict(dn, X_test)[1:], y_scaler)
pit_hist_dn = pit.pit_hist(pit.pit_gaussian(y_test, *y_dn))
pred_dn = method.predict(interpreter, pit_hist_dn)

plot_interpretation(pit_hist_dn, pred_dn)

#### Deep Ensemble

In [ ]:
checkpoint = torch.load("models/crimson-firebrand-43.pt")
hyperparams = checkpoint["hyperparams"]
de = method.DE(inputs=hyperparams["inputs"],
                 neurons=hyperparams["neurons"],
                 members=hyperparams["m"])
de.load_state_dict(checkpoint["model_state_dict"])
de

In [ ]:
y_de = inverse_transform(method.predict(de, X_test), y_scaler)
pit_hist_de = pit.pit_hist(pit.pit_gaussian(y_test, *y_de))
pred_de = method.predict(interpreter, pit_hist_de)

In [ ]:
plot_interpretation(pit_hist_de, pred_de)

#### Mixture density network

In [ ]:
checkpoint = torch.load("models/dainty-feather-49.pt")
hyperparams = checkpoint["hyperparams"]
mdn = method.MDN(inputs=9, neurons=hyperparams["neurons"], components=5)
mdn.load_state_dict(checkpoint["model_state_dict"])
mdn

In [ ]:
y_mdn = inverse_transform(method.predict(mdn, X_test), y_scaler)
pit_hist_mdn = pit.pit_hist(pit.pit_gaussian_mixture(y_test, *y_mdn))

In [ ]:
_, ax = plt.subplots()
plot.pit_hist(ax, pit_hist_mdn)